In [ ]:
# Getting data from aerospace database.
using CSV, DataFrames, Statistics, Random
include("../src/augment.jl")
include("../src/bin_to_leaves.jl")

In [ ]:
# Optional extra cores
using Distributed
addprocs(1)

In [ ]:
columns = ["Re", "thick", "M", "C_L"];
X = CSV.read("../data/airfoil/airfoil_X.csv", copycols=true, header=columns, delim=",");
Y = CSV.read("../data/airfoil/airfoil_Y.csv", copycols=true, header=["C_D"], delim=",");
# X = Matrix(X); Y = Matrix(Y);
Re = Array(range(10000, stop=35000, step=5000));
thick = [100,110,120,130,140,145];
M = [0.4, 0.5, 0.6, 0.7, 0.8, 0.9];
cl = Array(range(0.35, stop=0.70, step=0.05));

In [ ]:
# Testing augmentation
fns, fn_names = quad_fns(X);
print(fn_names)
fns, fn_names = bilinear_fns(X);
print(fns)

In [ ]:
# Now try augmentations
fns, fn_names = quad_fns(X);
X, fns = augment(X, fns, fn_names);
X, means, vars = normalize(X);
(train_X, train_y), (test_X, test_y) = IAI.split_data(:regression, seed=1, Matrix(X), Matrix(Y), train_proportion=0.8);
size(train_X)

In [ ]:
# Try optimal feature selection
lnr = IAI.OptimalFeatureSelectionRegressor()
grid = IAI.GridSearch(lnr, criterion=[:mse, :l1hinge, :l2hinge], sparsity=3:10)
IAI.fit!(grid, train_X, train_y, test_X, test_y, validation_criterion=:mse)

In [ ]:
# MSE errors of SVM/linear regression
lnr = IAI.get_learner(grid)
println("Training MSE:", 1 - IAI.score(lnr, train_X, train_y, criterion=:mse))
println("Test MSE:", 1- IAI.score(lnr, test_X, test_y, criterion=:mse))

In [ ]:
# Plotting results
predictions = IAI.predict(lnr, X)
using Plots
plt = Plots.plot(X[:,3], X[:,4], Y[:,1], seriestype=:scatter, markersize = 2)
plt = Plots.plot!(X[:,3], X[:,4], predictions, seriestype=:scatter, markersize = 2)
display(plt)
# Obviously not great...

In [ ]:
# What if we fit a tree over the augmented features?
lnr = IAI.OptimalTreeRegressor(random_seed=31415, max_depth=2, cp=0.001, minbucket=0.05, regression_sparsity=:all, 
    regression_lambda = 0.0001, regression_weighted_betas=true, fast_num_support_restarts=10)
IAI.fit!(lnr, train_X, train_y)

In [ ]:
lnr = IAI.get_learner(grid)

In [ ]:
# Plotting results
predictions = IAI.predict(lnr, X)
using Plots
plt = Plots.plot(X[:,3], X[:,4], Y[:,1], seriestype=:scatter, markersize = 2)
plt = Plots.plot!(X[:,3], X[:,4], predictions, seriestype=:scatter, markersize = 2)
display(plt)


In [ ]:
# MSE errors
println("Training MSE:", 1 - IAI.score(lnr, train_X, train_y, criterion=:mse))
println("Test MSE:", 1- IAI.score(lnr, test_X, test_y, criterion=:mse))

In [ ]:
size(X)